##  Use super learning for quality index prediction

I will start with explore estimation with different learners. From the characteristics of each leaner. I will gain understanding about the data set. These konwledge will help me to build super learner.

In [2]:
import numpy as np
import pandas as pd 

In [3]:
data=pd.read_csv("/Users/lurui/Desktop/Insight/evaluation.data.csv")

In [4]:
data_full=pd.read_csv("/Users/lurui/Desktop/Insight/overall.data.csv")

In [5]:
data_full=data_full.drop(["Unnamed: 0"],axis=1)

In [6]:
data_full.columns

Index(['Name', 'Speciality', 'Star Rating', 'Staff', 'Punctuality',
       'Helpfulness', 'Knowledge', 'comment', 'Rating Usefulness',
       'Review Date', 'Link', 'text_length', 'trim_comment', 'neg', 'neu',
       'pos', 'compound', 'polarity', 'subjectivity', 'Document_No',
       'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text',
       'Positive Sentiment', 'Negative Sentiment'],
      dtype='object')

In [7]:
data=data.drop(["Unnamed: 0"],axis=1)

In [8]:
data["Name"]=data_full["Name"]

In [9]:
data.head(10)

,Topic_Perc_Contrib,readability,richness,objectivity,Review Useful,text,Name
0,0.3777,8.73,35.000000,0.181667,0,"['think', 'extremely', 'care', 'attentive', 'n...",Dr. Natan Schleider
1,0.3602,8.71,78.493333,0.622619,0,"['friendly', 'punctual', 'super', 'email', 'la...",Dr. Natan Schleider
2,0.3879,8.84,84.000000,1.000000,0,"['experience', 'doctor', 'past_experiences', '...",Dr. Natan Schleider
3,0.3933,8.67,39.631589,0.500000,0,"['knowledgeable', 'friendly', 'manner', 'highl...",Dr. Natan Schleider
4,0.3933,8.87,112.000000,0.500000,0,"['knowledgeable', 'friendly', 'manner', 'highl...",Dr. Natan Schleider
5,0.3911,8.53,102.060000,0.000000,1,"['excellent', 'physician', 'exceptional', 'bed...",Dr. Natan Schleider
6,0.3678,7.40,80.920000,0.325000,1,"['see', 'schleider', 'extremely', 'thorough', ...",Dr. Natan Schleider
7,0.3980,8.22,15.000000,0.366667,1,"['fantastic', 'service', 'friendly', 'knowledg...",Dr. Natan Schleider
8,0.3659,10.17,68.040000,0.600000,1,"['work', 'care', 'patient', 'require', 'hour',...",Dr. Natan Schleider
9,0.3869,6.01,71.680000,0.400000,1,"['place', 'neat', 'really', 'enjoy', 'experien...",Dr. Natan Schleider


In [10]:
#### The data set is unbalanced
data['Review Useful'].value_counts()

0    9670
1    2859
Name: Review Useful, dtype: int64

In [11]:
### create balanced data by using re_sample

In [12]:
Rated_data=data[data['Review Useful']==1]

In [13]:
Unrated_data=data[data['Review Useful']==0]

In [14]:
Unrated_data=Unrated_data.reset_index()

In [15]:
Unrated_data=Unrated_data.drop(["index"],axis=1)

In [16]:
Unrated_data.head(10)

,Topic_Perc_Contrib,readability,richness,objectivity,Review Useful,text,Name
0,0.3777,8.73,35.000000,0.181667,0,"['think', 'extremely', 'care', 'attentive', 'n...",Dr. Natan Schleider
1,0.3602,8.71,78.493333,0.622619,0,"['friendly', 'punctual', 'super', 'email', 'la...",Dr. Natan Schleider
2,0.3879,8.84,84.000000,1.000000,0,"['experience', 'doctor', 'past_experiences', '...",Dr. Natan Schleider
3,0.3933,8.67,39.631589,0.500000,0,"['knowledgeable', 'friendly', 'manner', 'highl...",Dr. Natan Schleider
4,0.3933,8.87,112.000000,0.500000,0,"['knowledgeable', 'friendly', 'manner', 'highl...",Dr. Natan Schleider
5,0.4161,6.06,59.958724,0.640079,0,"['luckily', 'able', 'son', 'office', 'able', '...",Dr. Natan Schleider
6,0.4161,7.64,98.746667,0.388889,0,"['bad', 'physician', 'ever_encountere', 'inter...",Dr. George P. Liakeas
7,0.3493,6.07,42.000000,0.422222,0,"['please', 'lucky', 'primary', 'physician', 's...",Dr. George P. Liakeas
8,0.3333,5.11,54.880000,1.000000,0,['tell'],Dr. George P. Liakeas
9,0.3675,5.97,80.172199,0.402778,0,"['shocked', 'person', 'notice', 'safely', 'sur...",Dr. George P. Liakeas


In [17]:
import random 

In [18]:
select_index=random.sample(list(range(0,Unrated_data.shape[0])),4000)  # Choose 4000 elements

In [19]:
Unrated_data_sel=Unrated_data.iloc[select_index]

In [20]:
Unrated_data_sel=Unrated_data_sel.reset_index()

In [21]:
Unrated_data_sel=Unrated_data_sel.drop(["index"],axis=1)

In [22]:
### data use to model building
data_model=pd.concat([Unrated_data_sel,Rated_data])

In [23]:
data_model.shape

(6859, 7)

In [24]:
X=data_model[["Topic_Perc_Contrib","readability","richness","objectivity","text"]]
y=data_model["Review Useful"]

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x,preprocessor=lambda x: x, stop_words='english')

In [28]:
tfidf.fit_transform(x_train['text'])

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<5144x30 sparse matrix of type '<class 'numpy.float64'>'
	with 121310 stored elements in Compressed Sparse Row format>

In [29]:
X_train_Tfidf = tfidf.transform(x_train['text'])
X_test_Tfidf = tfidf.transform(x_test['text'])

In [30]:
### Get all Tfidf data 
X_all_Tfidf = tfidf.transform(X['text'])

In [31]:
Text_data=pd.DataFrame(list(X_all_Tfidf.toarray()))

In [32]:
Text_data=Text_data.reset_index()

In [33]:
Text_data.tail(10)

,index,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
6849,6849,0.325088,0.669035,0.325088,0.011917,0.011917,0.000000,0.016583,0.075605,0.091517,...,0.000000,0.195246,0.139151,0.122862,0.109471,0.016846,0.046716,0.041035,0.071422,0.000000
6850,6850,0.272065,0.674896,0.272065,0.067319,0.067319,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.068935,0.142922,0.138811,0.000000,0.095167,0.000000,0.000000,0.161386,0.000000
6851,6851,0.351757,0.709510,0.351757,0.005707,0.005707,0.000000,0.047654,0.066385,0.062615,...,0.000000,0.116888,0.151464,0.176529,0.045876,0.048410,0.044748,0.000000,0.068413,0.000000
6852,6852,0.318817,0.685421,0.318817,0.026296,0.026296,0.050519,0.109777,0.111221,0.028849,...,0.000000,0.134634,0.223309,0.108443,0.090584,0.037173,0.000000,0.000000,0.063040,0.000000
6853,6853,0.338242,0.703209,0.338242,0.015942,0.015942,0.000000,0.066552,0.067427,0.104936,...,0.042071,0.097946,0.152302,0.213664,0.073222,0.067608,0.020831,0.000000,0.057326,0.000000
6854,6854,0.289771,0.638950,0.289771,0.031867,0.031867,0.061221,0.000000,0.101088,0.034961,...,0.000000,0.163158,0.236792,0.459962,0.073184,0.000000,0.000000,0.000000,0.076395,0.000000
6855,6855,0.350768,0.718557,0.350768,0.011199,0.011199,0.000000,0.000000,0.082894,0.024573,...,0.000000,0.114679,0.142658,0.138554,0.025719,0.031664,0.058536,0.000000,0.026848,0.000000
6856,6856,0.326465,0.697711,0.326465,0.024855,0.024855,0.000000,0.069176,0.105128,0.027268,...,0.000000,0.101807,0.105538,0.051251,0.057081,0.070274,0.097437,0.000000,0.089380,0.000000
6857,6857,0.281190,0.627777,0.281190,0.034789,0.034789,0.066834,0.048411,0.183927,0.038166,...,0.000000,0.178117,0.036929,0.107600,0.039947,0.000000,0.045459,0.059897,0.083399,0.098171
6858,6858,0.299889,0.654647,0.299889,0.029682,0.029682,0.000000,0.082608,0.188312,0.195379,...,0.000000,0.212757,0.000000,0.183609,0.068165,0.000000,0.038785,0.000000,0.106735,0.000000


In [34]:
Num_data=data_model[['Topic_Perc_Contrib', 'readability', 'richness', 'objectivity']]

In [35]:
Num_data.columns

Index(['Topic_Perc_Contrib', 'readability', 'richness', 'objectivity'], dtype='object')

In [37]:
Num_data.head(10)

,Topic_Perc_Contrib,readability,richness,objectivity
0,0.3613,9.11,282.333333,0.100000
1,0.3360,1.08,49.373333,0.286111
2,0.3747,7.19,74.697778,0.400000
3,0.3750,8.58,73.960000,0.700000
4,0.3414,6.34,33.880000,0.450000
5,0.3779,6.64,95.203791,0.350000
6,0.3736,7.92,61.740000,0.100000
7,0.3547,6.69,16.940000,1.000000
8,0.3874,6.63,90.360000,1.000000
9,0.3501,6.13,29.229895,0.565000


In [38]:
Num_data=Num_data.reset_index()

In [43]:
Num_data=Num_data.drop(["index"],axis=1)

In [45]:
Num_data=Num_data.reset_index()

In [46]:
X_new=pd.merge(Text_data,Num_data,on="index")

In [47]:
X_new.shape

(6859, 35)

In [48]:
X_new=X_new.drop(["index"],axis=1)

In [49]:
X_new.shape

(6859, 34)

In [50]:
data_model.shape

(6859, 7)

In [51]:
result=list(data_model["Review Useful"])

In [52]:
X_new.insert(34,"reseult",result,True)

In [53]:
data_update=X_new

In [54]:
data_update.columns=["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10",
                 "X11","X12","X13","X14","X15","X16","X17","X18","X19","X20",
                 "X21","X22","X23","X24","X25","X26","X27","X28","29","X30","X31","X32","X33","X34","y"]

In [55]:
X=data_update.drop(["y"],axis=1)

In [56]:
y=data_update["y"]

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

from scipy import sparse

Train_features = sparse.hstack((Train_X_Tfidf, x_train[['Topic_Perc_Contrib', 'readability', 'richness', 'objectivity']])).tocsr()

Test_features= sparse.hstack((Test_X_Tfidf, x_test[['Topic_Perc_Contrib', 'readability', 'richness', 'objectivity']])).tocsr()

In [58]:
from sklearn.preprocessing import StandardScaler

In [61]:
scaler = StandardScaler().fit(x_train)

In [64]:
x_train_scaled=scaler.transform(x_train)

In [65]:
x_test_scaled=scaler.transform(x_test)

In [80]:
X_scaled=scaler.transform(X)

### Learner 1: Logistic regression with penalty 

In [66]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

In [67]:
logisticRegr.fit(x_train_scaled, y_train)

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
predictions_lg = logisticRegr.predict(x_test_scaled)

In [69]:
from sklearn.metrics import f1_score

In [70]:
f1_score(predictions_lg, y_test, average='micro')

0.570262390670554

In [74]:
## Add pernalty 

In [75]:
# Create regularization penalty space
penalty = ['l1', 'l2']

In [76]:
# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

In [77]:
# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [78]:
from sklearn.model_selection import GridSearchCV

In [79]:
clf = GridSearchCV(logisticRegr, hyperparameters, cv=5, verbose=0)

In [83]:
# Fit grid search
best_model = clf.fit(X_scaled, y)

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: 

In [84]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 1.0


In [85]:
logisticRegr_penalty = LogisticRegression(penalty ='l1',C=1.0)

In [86]:
logisticRegr_penalty.fit(x_train_scaled, y_train)

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [87]:
predictions_lg_pen = logisticRegr_penalty.predict(x_test_scaled)

In [88]:
f1_score(predictions_lg_pen, y_test, average='micro')

0.5720116618075802

### Learner 2: Random Forest Classfier 

In [89]:
from sklearn.ensemble import RandomForestClassifier

In [92]:
forest =RandomForestClassifier(random_state = 1)
forest.fit(x_train_scaled, y_train)

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [94]:
prediction_rf = forest.predict(x_test_scaled)

In [95]:
f1_score(prediction_rf, y_test, average='micro')

0.5941690962099125

In [96]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

In [97]:
hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)


In [98]:
gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)

In [99]:
bestF = gridF.fit(x_train_scaled, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 30.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 57.0min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 70.2min finished


In [103]:
bestF.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=15, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [104]:
forestOpt = RandomForestClassifier(random_state = 1, max_depth = 15, n_estimators = 300, min_samples_split = 5, min_samples_leaf = 1)

In [105]:
forestOpt.fit(x_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=15, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [106]:
prediction_rf_opt = forestOpt.predict(x_test_scaled)

In [107]:
f1_score(prediction_rf_opt, y_test, average='micro')

0.6169096209912537

### learner 3:  KNeighborsClassifier

In [109]:
from sklearn.neighbors import KNeighborsClassifier

In [110]:
from sklearn.model_selection import GridSearchCV

In [111]:
knn = KNeighborsClassifier()

In [112]:
param_grid = {'n_neighbors': np.arange(1, 25)}

In [113]:
#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn, param_grid, cv=5)

In [114]:
knn_gscv.fit(X_scaled, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [115]:
### Find the optimal number of paramter for knn
knn_gscv.best_params_

{'n_neighbors': 2}

In [116]:
knn_opt = KNeighborsClassifier(n_neighbors=2)

In [117]:
knn_opt.fit(x_train_scaled, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [118]:
prediction_knn_opt = knn_opt.predict(x_test_scaled)

In [119]:
f1_score(prediction_knn_opt, y_test, average='micro')

0.5918367346938775

### learner 4:  GaussianNB

In [143]:
from sklearn.naive_bayes import GaussianNB

In [144]:
bayes = GaussianNB()

In [145]:
bayes.fit(x_train_scaled, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [147]:
prediction_bayes = bayes.predict(x_test_scaled)

In [148]:
f1_score(prediction_bayes, y_test, average='micro')

0.5620991253644315

### learner 5:  SVM

In [149]:
from sklearn.svm import SVC

In [184]:
svc = SVC()

In [185]:
svc.fit(x_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [186]:
prediction_svc = svc.predict(x_test_scaled)

In [187]:
f1_score(prediction_svc, y_test, average='micro')

0.5865889212827988

In [188]:
param_grid = {'C': [0.1, 0.5,1],  
              'gamma': [1, 0.1, 0.01], 
              'kernel': ['rbf']}  

In [189]:
svc_grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

In [191]:
svc_grid.fit(X_scaled, y) 

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.583, total=   2.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.583, total=   2.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.4s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.583, total=   2.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.583, total=   3.6s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.584, total=   2.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.583, total=   2.1s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.583, total=   2.6s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.583, total=   2.6s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.583, total=   3.1s
[CV] C=0.5, gamma=1, kernel=rbf ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 0.5, 1], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [193]:
# print best parameter after tuning 
print(svc_grid.best_params_) 

{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}


### Super Learner

In [230]:
from mlens.ensemble import SuperLearner 

In [231]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from mlens.ensemble import BlendEnsemble

In [238]:
def build_ensemble(proba, **kwargs):
    """Return an ensemble."""
    estimators = [RandomForestClassifier(random_state = 1, max_depth = 15,
                                         n_estimators = 300, 
                                         min_samples_split = 5, 
                                         min_samples_leaf = 1),
                  SVC(probability=proba,C=0.1,gamma=0.01),
                  KNeighborsClassifier(n_neighbors=2),
                  LogisticRegression(penalty ='l1',C=10),
                  GaussianNB()
                  ]
    ensemble = BlendEnsemble(**kwargs)
    ensemble.add(estimators, proba=proba) 
    ensemble.add_meta(LogisticRegression())# Specify 'proba' her
    return ensemble

In [239]:
ensemble = build_ensemble(proba=True)

In [240]:
ensemble.fit(x_train_scaled, y_train)

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=None, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=...er=None)],
   n_jobs=-1, name='group-22', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=None, sample_size=20, scorer=None, shuffle=False,
       test_size=0.5, verbose=False)

In [241]:
preds = ensemble.predict(x_test_scaled)

In [242]:
f1_score(preds, y_test, average='micro')

0.6239067055393586

### Calibrating 

In [243]:
def build_ensemble_calib(proba, **kwargs):
    """Return an ensemble."""
    estimators = [RandomForestClassifier(random_state = 1, max_depth = 15,
                                         n_estimators = 300, 
                                         min_samples_split = 5, 
                                         min_samples_leaf = 1),
                  SVC(probability=proba,C=0.1,gamma=0.01),
                  KNeighborsClassifier(n_neighbors=2),
                  LogisticRegression(penalty ='l1',C=10),
                  GaussianNB()
                  ]
    ensemble = BlendEnsemble(**kwargs)
    ensemble.add(estimators, proba=proba) 
    return ensemble

In [244]:
ensemble_deduct=build_ensemble_calib(proba=True)

In [246]:
ensemble_deduct.fit(x_train_scaled,y_train)

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=None, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=...er=None)],
   n_jobs=-1, name='group-23', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=None, sample_size=20, scorer=None, shuffle=False,
       test_size=0.5, verbose=False)

In [248]:
### All data set 
X_all=data[["Topic_Perc_Contrib","readability","richness","objectivity","text"]]
y_all=data["Review Useful"]

In [249]:
X_all_Tfidf = tfidf.transform(X_all['text'])

In [250]:
Text_data_all=pd.DataFrame(list(X_all_Tfidf.toarray()))

In [257]:
Text_data_all=Text_data_all.reset_index()

In [252]:
Num_data_all=data[['Topic_Perc_Contrib', 'readability', 'richness', 'objectivity']]

In [255]:
Num_data_all=Num_data_all.reset_index()

In [258]:
X_new_all=pd.merge(Text_data_all,Num_data_all,on="index")

In [263]:
X_new_all=X_new_all.drop(["index"],axis=1)

In [264]:
X_new_all.columns=["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10",
                 "X11","X12","X13","X14","X15","X16","X17","X18","X19","X20",
                 "X21","X22","X23","X24","X25","X26","X27","X28","29","X30","X31","X32","X33","X34"]

In [265]:
X_new_all_scaled=scaler.transform(X_new_all)

In [267]:
preds=ensemble_deduct.predict(X_new_all_scaled)

In [268]:
### Get the score data set 
score_data=pd.DataFrame(preds)

In [272]:
score_data.columns=["L1","L2","L3","L4","L5","L6","L7","L8","L9","L10"]

In [273]:
score_data_sub=score_data[["L1","L3","L5","L7","L9"]]

In [276]:
score_data_sub["Y"]=y_all

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [277]:
score_data_sub

,L1,L3,L5,L7,L9,Y
0,0.544160,0.5,0.554085,0.751502,0.595319,0
1,0.477533,0.5,0.495980,0.530621,0.579446,0
2,0.537747,0.5,0.553744,0.569753,0.592625,0
3,0.399280,0.5,0.508398,0.704139,0.575706,0
4,0.409313,0.5,0.526952,0.743392,0.581907,0
...,...,...,...,...,...,...
12524,0.999639,0.5,0.668223,0.784041,0.598747,0
12525,0.660107,0.5,0.613164,0.760414,0.600831,0
12526,0.993295,0.5,0.632783,0.578141,0.576481,0
12527,0.659618,0.5,0.579238,0.527699,0.585716,0


In [278]:
lg = LogisticRegression()

In [279]:
lg.fit(score_data_sub.drop("Y",axis=1),score_data_sub["Y"])

/Applications/anaconda/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [280]:
quality_index=pd.DataFrame(lg.predict_proba(score_data_sub.drop("Y",axis=1))[:,1])

In [281]:
quality_index['Name']=data["Name"]

In [282]:
quality_index.columns=["Quality Index","Name"]

In [283]:
quality_index["Dominant_Topic"]=data_full['Dominant_Topic']

In [284]:
quality_index['comment']=data_full['comment']

In [285]:
quality_index.to_csv(r'quality_data.csv', encoding='utf8')

In [286]:
quality_index=quality_index.drop_duplicates()

In [287]:
quality_index_new=quality_index.groupby(["Name","Dominant_Topic"])

In [288]:
quality_index_new_1=quality_index_new.apply(lambda x: x.sort_values(["Quality Index"],ascending=False))

In [289]:
quality_index_new_2=quality_index_new_1.reset_index(drop=True)

In [290]:
quality_index_new_3=quality_index_new_2.groupby(["Name","Dominant_Topic"]).head(1)

In [291]:
quality_index_new_3

,Quality Index,Name,Dominant_Topic,comment
0,0.118398,Dr. Aaron N. Manson,1.0,My friend brought me to see Dr. Gartner for co...
4,0.084188,Dr. Aaron N. Manson,2.0,Dr. Manson is a very caring physician who has ...
6,0.869872,Dr. Adam D. Schaffner,1.0,I received injections (fillers) and suffered f...
17,0.943626,Dr. Adam D. Schaffner,2.0,I’m extremely happy with my results. I was wel...
24,0.324554,Dr. Adam D. Zeitlin,2.0,Really nice and I had an extremely pleasant vi...
...,...,...,...,...
11087,0.224042,Dr. Zachary Bregman,2.0,I have been a patient of Dr Bregman for over a...
11089,0.063921,Dr. Zhanna Tsukerman,1.0,"As of April 2019, she is under investigation w..."
11090,0.247556,Dr. Zhanna Tsukerman,2.0,Dr Tsukerman is a great phisician with excelle...
11094,0.002274,Dr. Zurab Abayev,0.0,"Very reliable and helpful. Pay me , I'll not g..."


In [293]:
quality_index_new_3=quality_index_new_3.reset_index()

In [294]:
quality_index_new_3.tail(10)

,index,Quality Index,Name,Dominant_Topic,comment
1230,11053,0.302724,Dr. William Schell,2.0,Very happy with my visit. Dr. Schell was on ti...
1231,11057,0.112434,Dr. Yael Freeman,2.0,Dr. Yael Freeman is the best dentist! We have ...
1232,11060,0.165952,Dr. Yiling Shiao,0.0,My wife and I traveled to NYC this long weeken...
1233,11061,0.308461,Dr. Yiling Shiao,2.0,I felt very much at ease under the care of Dr....
1234,11064,0.976549,Dr. Zachary Bregman,0.0,Now I've had plenty of doctors in the past but...
1235,11087,0.224042,Dr. Zachary Bregman,2.0,I have been a patient of Dr Bregman for over a...
1236,11089,0.063921,Dr. Zhanna Tsukerman,1.0,"As of April 2019, she is under investigation w..."
1237,11090,0.247556,Dr. Zhanna Tsukerman,2.0,Dr Tsukerman is a great phisician with excelle...
1238,11094,0.002274,Dr. Zurab Abayev,0.0,"Very reliable and helpful. Pay me , I'll not g..."
1239,11095,0.505061,Dr. Zurab Abayev,2.0,Dr. Abayev is one of the best doctors I've eve...


In [295]:
comment_data=quality_index_new_3

In [296]:
comment_data["Quality Index"].describe()

count    1240.000000
mean        0.478087
std         0.360506
min         0.002026
25%         0.137800
50%         0.327074
75%         0.882778
max         0.997981
Name: Quality Index, dtype: float64

In [297]:
import os

In [298]:
os.chdir('/Users/lurui/Desktop/Insight')

In [299]:
comment_data.to_csv(r'comment_data.csv', encoding='utf8')